In [ ]:
import requests
import numpy as np
import pandas as pd

import time
from json.decoder import JSONDecodeError

from google.oauth2 import service_account
import pandas_gbq

from datetime import date

In [ ]:
def get_refresh_token():
  parms = {"refreshtoken":"hRv3j2VHiixfpBRRWi5SA6zx9acCAIB1rXY8ZcanU9nETNxj3o52giWbYVUO8hCU"}
  token_url = 'https://api.chartmetric.com/api/token'
  token_res = requests.post(token_url, data=parms)
  refresh_token = token_res.json()['token']
  # print(token_res.json())
  return refresh_token

In [ ]:
def get_artist_data(refresh_token):
  for min_range in range(1,201,200): #fetches 200 records, use cm_artist_rank to fetch top 200 artists, change 200 to get more number of records
    max_range = min_range + 199
    headers= {'Authorization' : 'Bearer '+refresh_token}
    url = 'https://api.chartmetric.com/api/artist/cm_artist_rank/list?min='+str(min_range)+'&max='+str(max_range)+'&limit=200'
    res = requests.get(url, headers=headers)
    time.sleep(1)
    if min_range == 1: # put the same min value of for loop here
      new_df = pd.DataFrame(res.json()['obj']['data'])
    else:
      df = pd.DataFrame(res.json()['obj']['data'])
      new_df = pd.concat([new_df,df])
    time.sleep(1)
  return new_df

In [ ]:
refresh_token = get_refresh_token()
df = get_artist_data(refresh_token)

In [ ]:
def get_age_level_data(age_list, age_bucket):
    for data in age_list:
      for k,v in data.items():
        if v == age_bucket:
          return round(float(data['male']) + float(data['female']),2)

def get_platform_data(id_list, refresh_token, platform):
    stats_result = {'artist_id':[],
                     platform+'_followers':[],
                     platform+'_Genz_followers_percentage' : [],
                     platform+'_avg_likes_per_post':[],
                     platform+'_avg_commments_per_post':[]}

    for id in id_list:
     #print(id)
     headers= {'Authorization' : 'Bearer '+refresh_token}
     stats_url = 'https://api.chartmetric.com/api/artist/'+id+'/'+platform.lower()+'-audience-stats'
     stats_res = requests.get(stats_url, headers=headers)

     if platform == 'TikTok':
      if stats_res.reason == 'Not Found':
        pass
      elif len(stats_res.json()['obj']) == 8 and stats_res.json()['obj']['audience_genders_per_age'] != []:
        stats_result['artist_id'].append(id)
        stats_result[platform+'_followers'].append(stats_res.json()['obj']['followers'])
        stats_result[platform+'_Genz_followers_percentage'].append(get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '13-17') +
                                                          get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '18-24'))
        stats_result[platform+'_avg_likes_per_post'].append(stats_res.json()['obj']['avg_likes_per_post'])
        stats_result[platform+'_avg_commments_per_post'].append(stats_res.json()['obj']['avg_commments_per_post'])

     elif platform == 'Instagram':
      if stats_res.reason == 'Not Found':
        pass
      elif stats_res.reason == 'OK' and len(stats_res.json()['obj']) == 13 and stats_res.json()['obj']['audience_genders_per_age'] != []:
        stats_result['artist_id'].append(id)
        stats_result[platform+'_followers'].append(stats_res.json()['obj']['followers'])
        stats_result[platform+'_Genz_followers_percentage'].append(get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '13-17') +
                                                          get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '18-24'))
        stats_result[platform+'_avg_likes_per_post'].append(stats_res.json()['obj']['avg_likes_per_post'])
        stats_result[platform+'_avg_commments_per_post'].append(stats_res.json()['obj']['avg_commments_per_post'])

      #Youtube audience data, object length = 9
     elif platform == 'Youtube':
      if stats_res.reason == 'Not Found':
        pass
      elif stats_res.reason == 'OK' and len(stats_res.json()['obj']) == 9 and stats_res.json()['obj']['audience_genders_per_age'] != []:
        stats_result['artist_id'].append(id)
        stats_result[platform+'_followers'].append(stats_res.json()['obj']['subscribers'])
        stats_result[platform+'_Genz_followers_percentage'].append(get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '13-17') +
                                                          get_age_level_data(stats_res.json()['obj']['audience_genders_per_age'], '18-24'))
        stats_result[platform+'_avg_likes_per_post'].append(stats_res.json()['obj']['avg_likes_per_post'])
        stats_result[platform+'_avg_commments_per_post'].append(stats_res.json()['obj']['avg_commments_per_post'])

    return stats_result


In [ ]:
def get_tiktok_creations_data(id_list,refresh_token):# creations data, how many people used the video/audio of the creator
  tracks_result = {'artist_id':[],
                 'TikTok_number_of_top_tracks':[],
                 'TikTok_total_creations_for_top_n_tracks':[],
                 'TikTok_total_views_for_top_n_tracks':[]}
  for id in id_list:
    #print(id)
      headers= {'Authorization' : 'Bearer '+refresh_token}
      url = 'https://api.chartmetric.com/api/artist/'+id+'/top-tracks/tiktok?limit=30'
      res = requests.get(url, headers=headers)
      top_tracks = {}
      top_views= {}
      top_likes={}
      top_comments={}
      i = 1
      tracks_result['artist_id'].append(id)
      for track in res.json()['obj']:
        if track['posts'] is None or track['views'] is None:
          pass
        elif len(top_tracks.values()) < 10:
          # print(track['name'],track['posts'])
          # print(top_tracks)
          if track['name'] in top_tracks.keys():
            top_tracks[track['name']+str(i)] = track['posts']
            top_views[track['name']+str(i)] = track['views']
            top_likes[track['name']+str(i)] = track['likes']
            top_comments[track['name']+str(i)]= track['comments']
            i+=1
          else:
            top_tracks[track['name']] = track['posts']
            top_views[track['name']] = track['views']
            top_likes[track['name']] = track['likes']
            top_comments[track['name']]= track['comments']

      tracks_result['TikTok_number_of_top_tracks'].append(len(top_tracks.values()))
      tracks_result['TikTok_total_creations_for_top_n_tracks'].append(sum(top_tracks.values()))
      tracks_result['TikTok_total_views_for_top_n_tracks'].append(sum(top_views.values()))

  return tracks_result

In [ ]:
def tiktok_call(artist_df, i, refresh_token, final_tiktok_df):
  result = get_platform_data(list(artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')), refresh_token, 'TikTok')
  tiktok_df = pd.DataFrame(result)
  tiktok_df['TikTok_Genz_followers'] = round((tiktok_df['TikTok_Genz_followers_percentage'] * (tiktok_df['TikTok_followers']/100)),2)
  time.sleep(1)
  result2 = get_tiktok_creations_data(list(artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')), refresh_token)
  track_df = pd.DataFrame(result2)
  track_df['TikTok_avg_creations_for_top_n_tracks'] = round(track_df['TikTok_total_creations_for_top_n_tracks']/track_df['TikTok_number_of_top_tracks'])
  track_df['TikTok_avg_views_for_top_n_tracks'] = round(track_df['TikTok_total_views_for_top_n_tracks']/track_df['TikTok_number_of_top_tracks'])
  track_df = pd.merge(track_df,artist_df[['chartmetric_artist_id','name']].astype('str'),left_on=['artist_id'],right_on=['chartmetric_artist_id'],how='left')
  merge_tiktok_df = pd.merge(tiktok_df,track_df,on=['artist_id'],how='right')
  merge_tiktok_df['TikTok_avg_engagements_per_post'] = merge_tiktok_df['TikTok_avg_likes_per_post'] + merge_tiktok_df['TikTok_avg_commments_per_post']
  merge_tiktok_df = merge_tiktok_df[['artist_id','name','TikTok_followers','TikTok_Genz_followers','TikTok_Genz_followers_percentage',
                                     'TikTok_avg_engagements_per_post','TikTok_number_of_top_tracks',
                                     'TikTok_total_creations_for_top_n_tracks','TikTok_avg_creations_for_top_n_tracks',
                                     'TikTok_total_views_for_top_n_tracks','TikTok_avg_views_for_top_n_tracks']]

  merge_tiktok_df.fillna("",inplace=True)
  final_tiktok_df = pd.concat([final_tiktok_df,merge_tiktok_df], axis=0)
  return final_tiktok_df

In [ ]:
def instagram_call(artist_df, i, refresh_token, final_insta_df):
  result = get_platform_data(list(artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')), refresh_token, 'Instagram')
  insta_df = pd.DataFrame(result)
  insta_df['Instagram_Genz_followers'] = round((insta_df['Instagram_Genz_followers_percentage'] * (insta_df['Instagram_followers']/100)),2)
  time.sleep(1)
  merge_insta_df = pd.merge(insta_df,artist_df[['chartmetric_artist_id','name']].astype('str'),left_on=['artist_id'],right_on=['chartmetric_artist_id'],how='left')
  merge_insta_df['Instagram_avg_engagements_per_post'] = merge_insta_df['Instagram_avg_likes_per_post'] + merge_insta_df['Instagram_avg_commments_per_post']
  merge_insta_df = merge_insta_df[['artist_id','name','Instagram_followers','Instagram_Genz_followers','Instagram_Genz_followers_percentage',
                                   'Instagram_avg_engagements_per_post']]
  merge_insta_df.fillna("",inplace=True)
  final_insta_df = pd.concat([final_insta_df,merge_insta_df], axis=0)
  return final_insta_df

In [ ]:
def youtube_call(artist_df, i, refresh_token, final_youtube_df):
  result = get_platform_data(list(artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')), refresh_token, 'Youtube')
  youtube_df = pd.DataFrame(result)
  youtube_df['Youtube_Genz_followers'] = round((youtube_df['Youtube_Genz_followers_percentage'] * (youtube_df['Youtube_followers']/100)),2)
  time.sleep(1)
  merge_youtube_df = pd.merge(youtube_df,artist_df[['chartmetric_artist_id','name']].astype('str'),left_on=['artist_id'],right_on=['chartmetric_artist_id'],how='left')
  merge_youtube_df['Youtube_avg_engagements_per_post'] = merge_youtube_df['Youtube_avg_likes_per_post'] + merge_youtube_df['Youtube_avg_commments_per_post']
  merge_youtube_df = merge_youtube_df[['artist_id','name','Youtube_followers','Youtube_Genz_followers','Youtube_Genz_followers_percentage',
                                   'Youtube_avg_engagements_per_post']]
  merge_youtube_df.fillna("",inplace=True)
  final_youtube_df = pd.concat([final_youtube_df,merge_youtube_df], axis=0)
  return final_youtube_df

In [ ]:
def final_merge_call(artist_df, i, final_tiktok_df, final_insta_df, final_youtube_df):
  artist_df['chartmetric_artist_id'].iloc[i:i+1] = artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')
  final_df = pd.merge(artist_df.iloc[i:i+1],final_tiktok_df,left_on=['chartmetric_artist_id'],right_on=['artist_id'],how='left')
  final_df = pd.merge(final_df,final_insta_df,left_on=['chartmetric_artist_id'],right_on=['artist_id'],how='left')
  final_df = pd.merge(final_df, final_youtube_df, left_on = ['chartmetric_artist_id'], right_on=['artist_id'], how='left')
  final_df = final_df.drop(['chartmetric_artist_id','artist_id_x','artist_id_y','name_y'],axis=1)
  final_df.rename(columns={'name_x' : 'name','code2' : 'country_code', 'Youtube_followers' : 'Youtube_subscribers',
                           'Youtube_Genz_followers' : 'Youtube_Genz_subscribers', 'Youtube_Genz_followers_percentage' : 'Youtube_Genz_subscribers_percentage'},inplace=True)
  final_df.drop_duplicates(inplace=True)
  final_df.fillna('',inplace=True)
  return final_df

In [ ]:
from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
def main_call():
  refresh_token = get_refresh_token()
  artist_df = get_artist_data(refresh_token)
  artist_df.drop(['cpp_rank','rank_eg','rank_fb','spotify_popularity','spotify_listeners_to_followers_ratio',
                  'spotify_followers_to_listeners_ratio','deezer_fans','wikipedia_views','soundcloud_followers','bandsintown_followers',
                  'sp_where_people_listen','signed','tiktok_followers','tiktok_likes','youtube_subscribers', 'instagram_followers'],1,inplace=True)
  start = 0
  end= 3 #len(artist_df)
  counter=0
  final_tiktok_df = pd.DataFrame()
  final_insta_df = pd.DataFrame()
  final_youtube_df = pd.DataFrame()
  final_df_list = []

  for i in range(start,end):
    counter+=1
    final_tiktok_df = tiktok_call(artist_df,i,refresh_token, final_tiktok_df)
    time.sleep(1)
    final_insta_df = instagram_call(artist_df,i,refresh_token, final_insta_df)
    time.sleep(1)
    final_youtube_df = youtube_call(artist_df,i,refresh_token, final_youtube_df)
    time.sleep(1)
    merged_df = final_merge_call(artist_df,i, final_tiktok_df, final_insta_df, final_youtube_df)
    final_df_list.append(merged_df)
    print(str(counter),end=" ")
  final_df = pd.concat(final_df_list, ignore_index=True)
  final_df = final_df.loc[:, ~final_df.columns.duplicated()]
  return final_df
final_df = main_call()

<ipython-input-135-6130213a43f5>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  artist_df.drop(['cpp_rank','rank_eg','rank_fb','spotify_popularity','spotify_listeners_to_followers_ratio',
<ipython-input-132-18b74d7c1b75>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_df['chartmetric_artist_id'].iloc[i:i+1] = artist_df['chartmetric_artist_id'].iloc[i:i+1].astype('str')
<ipython-input-132-18b74d7c1b75>:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'name_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(final_df, final_youtube_df, left_on = ['chartmetric_artist_id'], right_on=['artist_id'], how='left')


1 

<ipython-input-132-18b74d7c1b75>:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'name_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(final_df, final_youtube_df, left_on = ['chartmetric_artist_id'], right_on=['artist_id'], how='left')


2 3 

<ipython-input-132-18b74d7c1b75>:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'name_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(final_df, final_youtube_df, left_on = ['chartmetric_artist_id'], right_on=['artist_id'], how='left')


In [ ]:
df_mod = final_df.drop_duplicates(subset=['artist_id'], keep='first')
df_mod = df_mod[df_mod['artist_id'] != ""]

In [ ]:
df_mod['Snapshot_Date'] = date.today()

In [ ]:
project_id = 'tech-cali-b2c'
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/tech-cali-b2c-72b3e690e309-Compute-Engine.json')

dataset_id = 'CE_External_Source'
table_name = 'Chartmetric_Raw'

df_mod = df_mod.astype(str)

# Write the dataframe to BigQuery: use if_exists='replace' for the first time, then use if_exists='append' for subsequent data pulls
pandas_gbq.to_gbq(df_mod, f'{dataset_id}.{table_name}', project_id=project_id, if_exists='replace', credentials=credentials, progress_bar=True)

print('Data moved to BigQuery successfully!')

100%|██████████| 1/1 [00:00<00:00, 1745.44it/s]

Data moved to BigQuery successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
